In [7]:
# Import the MultiverseFanova helper class from analysis.py (Simson et al.)
from analysis import MultiverseFanova
import pandas as pd

In [11]:
# Ensure 'ambiguity' column exists as 1 - Frac_NonAmbiguous
all_metrics_df["ambiguity"] = 1 - all_metrics_df["Frac_NonAmbiguous"]

In [12]:
# List of unique subgroups to analyze
subgroups = all_metrics_df["Subgroup"].unique()

In [13]:
# Loop through each subgroup and run FANOVA
for subgroup in subgroups:
    df_sub = all_metrics_df[all_metrics_df["Subgroup"] == subgroup].copy()
    
    # Use the analytical decisions as categorical inputs
    X = df_sub[["feature_set", "model", "threshold_policy"]]
    y = df_sub["ambiguity"]

    # Run FANOVA
    fanova = MultiverseFanova(X_choices=X, y_values=y)
    importance_df = fanova.quantify_individual_importance()
    importance_df.sort_values(by="individual importance", ascending=False, inplace=True)
    
    # Print results
    print(f"\nFANOVA for subgroup: {subgroup}")
    print(importance_df["individual importance"].apply(lambda x: f"{100 * x:.2f}%"))

KeyError: "None of [Index(['feature_set', 'model', 'threshold_policy'], dtype='object')] are in the [columns]"